### Deriving the equation for the shape of a string under load
Begin by considering a problem: *Suppose you have string. Given that it is supported at its ends, is acted on by the force of gravity, and there is a known tension applied to the string, what will be its shape?*

This question will lead us to develop a simple differential equation. Solving this equation will introduce us to a pair of solution techniques - *finite difference and finite element methods*.

An infinitesimal force diagram of a length of string under tension yields two relations for the forces - one along the string, or longitudinal, and the other perpendicular to the string, or transverse. The force balances that result are seen by inspecting the triangles formed at the ends of the small segment of the string. They are

$T(x) \cos(\theta_1) = T(x+\Delta x)\cos(\theta_2) \equiv T$

$T(x) \sin(\theta_1) = T(x+\Delta x)\sin(\theta_2) + f\Delta x$

The first equation tells us the longitudinal tension, $T$ is the same everywhere in the string. Applying this definition expressed in the first equation, gives

$T(x+\Delta x) = \frac{T}{\cos(\theta_2)}$ and $T(x) = \frac{T}{\cos(\theta_1)}$ 

which, when substituted into the second, transverse force balance gives

$T\tan(\theta_1) = T \tan(\theta_2) + f\Delta x$

Recalling that slope is rise over run, or in terms of a triangle opposite over adjacent (which is the tangent) then substituting $\tan(\theta_1) = u'(x)$ and $\tan(\theta_2) = u'(x+\Delta x)$ gives after rearranging: 

$T\frac{u'(x+\Delta x) - u'(x)}{\Delta x} + f = 0$

From this expression, identify the discrete forward difference approximation of the derivative. As $\Delta x$ goes to zero, this becomes exact, and the equation describing the deformation of the string is

$T\frac{d^2u}{dx^2} = -f(x)$

where the tension on the string is $T$ and the load on the string is $f(x)$. The focus of the remainder of the lesson is how this equation can be solved numerically.

### Numerical solutions to the equation

At this point, the critical change is from differential equation to system of linear equations. This entails working backwards through what we just did, returning to those discrete deriviates. However, from past experience, we know that the central difference will be more accurate than the forward difference. The finite difference formula for the second derivative is found by taking the central difference twice. It is

$\frac{d^u}{dx^2} = \frac{u(x+h) - 2u(x) + u(x-h)}{h^2}$.

Returning to the soution of the differential equation, if we represent space as a series of $x_i$, where $i$ specifies position $x = x_0+ih$, then we can rewrite the equation for the unknown function $u$ as

$u_{i-1} - 2u_i + u_{i+1} = \frac{f_i h^2}{T}$

Where $f_i = f(x_i)$. Beginning at $x_0$, which correponds to $u_0 = u(x_0)$, a set of equations is formed. To simplify, define $g_i = \frac{f_i h^2}{T}$. They look like:

$-2u_1 + u_2 = g_1 - u_0$

$u_1 - 2u_2 + u_3 = g_2$

$u_2 - 2u_3 + u_4 = g_3$

$\vdots = \vdots$

$u_{N-3} - 2u_{N-2} + u_{N-1} = g_{N-2}$

$u_{N-2} - 2u_{N-1} = g_{N-1} - u_N$


Hopefully, you see the above for what it is - a system of linear equations. The $u_i$ form a vector of unknowns, which can be multiplied by a matrix of coefficients for the unknowns. The right-hand side is a vector of values derived from the load and tension on the string. This is more compactly written in matrix form as

$\begin{bmatrix}
    -2 &  1 &   &  & \\
    1  & -2 & 1 &  & \\
    \ddots& \ddots&\ddots &\ddots &\ddots      \\
     &   &  1 & -2 & 1 \\
     &  & & 1 & -2
\end{bmatrix} 
\begin{bmatrix}
u_1\\
u_2\\
\vdots\\
u_{N-2}\\
u_{N-1}
\end{bmatrix}=
\begin{bmatrix}
g_1 - u_0\\
g_2\\
\vdots\\
g_{N-2}\\
g_{N-1} - u_N
\end{bmatrix}
$

Or, the matrix equation, where terms correspond to the left to right reading of the previous equation:

$\mathbf{A}\mathbf{u} = \mathbf{b}$

What follows is a guided example to coding this solution. The technique applied is called the *finite differnce method* for solving a partial differential equation. Never mind that in this example it happens to be an ordinary differential equation, we'll expand the method soon enough. Go through what follows and in each place you find an XXX, try to replace it with functioning code.

In [7]:
from scipy.linalg import solve       # SciPy linear eqn solver
import numpy as np, matplotlib.pyplot as plt

N, u0, uN= 20, 0.0, 0.0       # number of intervals, boundary values
x = np.linspace(u0, uN ,N)# grid, recall how a uniform spacing of N+1 points can be done
h, T, f = ((uN - u0) / N) , 1.0, -1.0       # bin size, tension, load

# Create the A matrix, study the documentation for diag.
A = np.diag([-2] * N)                 # diagonal 
A += np.diag([1] * (N - 1), 1) + np.diag([1] * (N - 1), -1) # off diagonals

b = np.array(A * x)                       # b matrix, study the derivation to determine the values
b[0], b[-1] = 0, 0                  # boundary values, just in case

u = solve(A, b)                         # solve     
u = np.insert(u, [0, N-1], [0, N - 1])    # insert BV at 1st and last pos 

plt.plot(x, u, label='$f=-1$'),  plt.legend(loc='center')   # legend 
plt.xlabel('$x$'), plt.ylabel('$u$'), plt.show();

                                                 

ValueError: x and y must have same first dimension, but have shapes (20,) and (402,)

### Finite differencing exercises

Now that you've completed the example code, try and accomplish the following:

1. Run the code again for $f=-2$ produce a plot that includes the curves for $f=-1$ and $f=-2$.
2. Solve for a load that increases linearly with distance from the origin; $f=-2x$. 
3. Solve for a quadratic load, $f(x) = -3x^2$. Try and anticipate the result before simply running the code. Plot the result on the same axes as the previous two problems.